In [1]:
import numpy as np
import pandas as pd
import cv2
import os

In [2]:
def find_coords(img, path):
    
    img1 = dirname_1+"\\"+img
    image = cv2.imread(img1)
    cropped_img = image[765:3598, 1008:2770]
    
    # wyskalowanie zdjęć
    scale_percent = 30
    width = int(cropped_img.shape[1] * scale_percent / 100)
    height = int(cropped_img.shape[0] * scale_percent / 100)
    size = (width, height)
    resized = cv2.resize(cropped_img, size)
    
    # przygotowanie do detekcji
    gray_resized = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    blured_gray_resized = cv2.medianBlur(gray_resized, 5)
    
    # detekcja okręgów
    circles = cv2.HoughCircles(blured_gray_resized,cv2.HOUGH_GRADIENT, 1.1, minDist=150, param1=50, param2=30, minRadius=1, maxRadius=22)

    # rysowanie wykrytego okręgu oraz jego punktu centralnego
    circles = np.uint16(np.around(circles))
    for i in circles[0, :]:
        cv2.circle(resized,(i[0], i[1]), i[2], (0,255,0), 2)
        cv2.circle(resized,(i[0], i[1]), 0, (0,0,255), -1)
    
    pixel_x = i[1]
    pixel_y = i[0]
    
    coord_x = 845 - pixel_x
    coord_y = 526 - pixel_y
    
    ratio = round(849/370, 2)
    
    coord_x /= ratio
    coord_y /= ratio
    
    x = img.split(".")
    i = x[0]

    
    
    coordinates = {"Nr obserwacji": int(i),"X": round(coord_x, 0), "Y": round(coord_y, 0)}
    coordinates = pd.Series(data=coordinates)
    
    cv2.imwrite(f"C:\\Users\\stadn\\Desktop\\_robot_dane_operator\\operatorzy_wykryte\\{img}", resized)
    
    return coordinates

In [ ]:
for i in range(1, 6):
    dirname_1 = "C:\\Users\\stadn\\Desktop\\_robot_dane_operator\\operator"
    results = pd.DataFrame()
    dirname_1 += str(i)
    for img in os.listdir(dirname_1):
        find_coords(img, dirname_1)
        results = results.append(find_coords(img, dirname_1), ignore_index=True)

    results = results.set_index("Nr obserwacji")
    results = results.sort_values(by="Nr obserwacji")
    results.to_excel(f"C:\\Users\\stadn\\Desktop\\_robot_dane_operator\\results{i}.xlsx")
    